# 5. Temporal Point Process (TPP)

In [ ]:
# Common imports 
import cvxpy as CVX
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erf

import sys
print(sys.version)

## 1. Homogeneous Poisson Process


In [ ]:
def sample_homogeneous(mu, t_prev):
    return np.random.exponential(scale=1/mu) + t_prev

## 2. Inhomogeneous Poisson Process

In [ ]:
def g_function_eval(theta_list, tau_list, beta,t):
    exp_term = np.exp(-beta*np.square(t - tau_list))
    return np.sum(np.multiply(theta_list,exp_term))

def find_max_g_function(T,theta_list, tau_list, beta, npoints=10000):
    max_g = 0
    for t in np.linspace(0, T, npoints):
        g_tmp = g_function_eval(theta_list, tau_list, beta,t)
        max_g = g_tmp if g_tmp > max_g else max_g
    
    return max_g
        
    

In [ ]:
def sample_inhomogeneous(theta_list, tau_list, beta, T, Nev, seed=None):
    """Generates a sample of a Hawkes process until one of the following happens:
      - The next generated event is after T
      - Nev events have been generated.

    Returns: a tuple with the event times and the last generated time.
    """
    np.random.seed(seed)

    # First event is generated just as for a normal Poisson process.

    tev = np.zeros(Nev)
    n = 0
    g_max = find_max_g_function(T, theta_list, tau_list, beta)
    next_arrival_time = sample_homogeneous(g_max, 0)
    tev[n] = next_arrival_time

    # Generate the next events
    n += 1
    while n < Nev:
        next_arrival_time = sample_homogeneous(g_max, next_arrival_time)

        if next_arrival_time < T:
            u = np.random.rand()
            g_t = g_function_eval(theta_list, tau_list, beta, t=next_arrival_time)

            if u <= g_t / g_max:
                tev[n] = next_arrival_time
                n += 1
        else:
            break

    tev = tev[0:n]

    if n == Nev:
        Tend = tev[-1]
    else:
        Tend = T

    return tev, Tend

In [ ]:
def evaluate_survival_integral(T, tau, beta):
    sqrt_beta = np.sqrt(beta)
    sqrt_pi = np.sqrt(np.pi)
    numerator = sqrt_pi* (erf(tau*sqrt_beta) + erf(sqrt_beta*(T-tau)))
    
    return numerator/(2*sqrt_beta)

In [ ]:
def preprocess_events_inhomogeneous(tev, T, beta, tau_list):
    N_events = len(tev)
    N_kernels = len(tau_list)
    cte_ij = np.zeros([N_events, N_kernels], dtype=float)
    K_j = evaluate_survival_integral(T, tau_list, beta)

    for i, tev_i in enumerate(tev):
        cte_ij[i,:] = np.exp(-beta * np.square(tev_i - tau_list))

    return cte_ij, K_j

### Play with the Inhomogeneous Poisson Process

In [ ]:
def plot_inhomogeneous(tev, theta_list, tau_list, beta, T, resolution):
    tvec = np.arange(0, T, step=T / resolution)
    
    colorLambda = ['r--', 'k--', 'g--', 'm--', 'c--']
    colorEv = ['r+', 'k+', 'g+', 'm+', 'c+']

    for i, tev_i in enumerate(tev): # We iterate over the realizations of the Hawkes Process
        n = -1
        l_t = np.zeros(len(tvec))

        for t in tvec:
            n += 1
            l_t[n] = g_function_eval(theta_list, tau_list, beta, t=t)

        plt.plot(tvec, l_t, colorLambda[i % len(colorLambda)])
        plt.plot(tev[i], np.zeros(len(tev[i])), colorEv[i % len(colorEv)])

In [ ]:
# Simulation time
T = 10

# Maximum number of events per realization
maxNev = 100

# Number of samples to take
Nsamples = 50

# g function parameters parameter
theta_list = np.array([12, 10])
tau_list= np.array([2, 8])
beta = 2

In [ ]:
tev = [None] * Nsamples
Tend = [None] * Nsamples
C_ij = [None] * Nsamples
K_j = np.zeros([Nsamples, len(tau_list)])
for n in range(Nsamples):
    tev[n], Tend[n] = sample_inhomogeneous(theta_list, tau_list, beta, T, maxNev)
    C_ij[n], K_j[n] = preprocess_events_inhomogeneous(tev[n], Tend[n], beta, tau_list)

In [ ]:
for n, tev_n in enumerate(tev):
    print('Sequence ({}) | len: {}'.format(n, len(tev_n)))

In [ ]:
plot_inhomogeneous(tev, theta_list, tau_list, beta, T, resolution=10000.0)
plt.ion()  # Make the plot interactive
plt.show()  # Show the plot. May not be needed in IPython

Find the ML estimate of the parameters

In [ ]:
def inohomogeneous_log_lik(T, theta_opt, C_ij_n, K_j_n, for_cvx=False):
    # The implementation has to be different for CVX and numpy versions because
    # CVX variables cannot handle the vectorized operations of Numpy  like
    # np.sum and np.log.

    L = 0
    for n, C_ij in enumerate(C_ij_n):
        K_j = K_j_n[n]
        for i in range(C_ij.shape[0]):
            theta_cij = [ th*c for th, c in zip(theta_opt, C_ij[i])]
            L += CVX.sum(CVX.log(CVX.sum(theta_cij)))
            
        theta_kj = [ th*k for th, k in zip(theta_opt, K_j)]
        L -= CVX.sum(theta_kj)

    return L

In [ ]:
theta_opt = [CVX.Variable() for _ in range(len(theta_list))]
constraints = [th_op >= 0 for th_op in theta_opt]

complete_log_likelihood = inohomogeneous_log_lik(Tend, theta_opt, C_ij, K_j, for_cvx=True)

prob = CVX.Problem(CVX.Maximize(complete_log_likelihood), constraints=constraints)

result = prob.solve(verbose=True, max_iters=5000, abstol=1e-10)

In [ ]:
error_theta = [ th_op.value - theta_list[i] for i, th_op in enumerate(theta_opt)]

print('error_theta = {}'.format(error_theta))

In [ ]:
print('Theta real: {}'.format(theta_list))
print('Theta est: {}'.format([ float(th_op.value)  for th_op in theta_opt]))

## 3. Hawkes Process

In [ ]:
def intensity_hawkes_exp_eval(mu, alpha, w, history, t):
    return mu + alpha * np.sum(np.exp(-w * (t - history)))

In [ ]:
def sample_hawkes(lambda_0, alpha_0, w, T, Nev, seed=None):
    """Generates a sample of a Hawkes process until one of the following happens:
      - The next generated event is after T
      - Nev events have been generated.

    Returns: a tuple with the event times and the last generated time.
    """

    np.random.seed(seed)

    # First event is generated just as for a normal Poisson process.

    tev = np.zeros(Nev)
    n = 0
    lambda_star = lambda_0
    next_arrival_time = sample_homogeneous(lambda_star, 0)
    tev[n] = next_arrival_time

    # Generate the next events
    n += 1
    while n < Nev:
        next_arrival_time = sample_homogeneous(lambda_star, next_arrival_time)

        if next_arrival_time < T:
            u = np.random.rand()
            lambda_s = intensity_hawkes_exp_eval(lambda_0, alpha_0, w, history=tev[0:n], t=next_arrival_time)

            if u <= lambda_s / lambda_star:
                tev[n] = next_arrival_time
                n += 1
                lambda_star =  lambda_s + alpha_0
        else:
            break

    tev = tev[0:n]  

    if n == Nev:
        Tend = tev[-1]
    else:
        Tend = T

    return tev, Tend

In [ ]:
def plot_hawkes(tev, l_0, alpha_0, w, T, resolution):
    tvec = np.arange(0, T, step=T / resolution)

    # We can obtain the expected value using the Laplace transform
    mu_t = (np.exp((alpha_0 - w) * tvec) + w * (1.0 / (alpha_0 - w)) *
            (np.exp((alpha_0 - w) * tvec) - 1)) * l_0

    plt.plot(tvec, mu_t, 'b-', linewidth=1.5)

    colorLambda = ['r--', 'k--', 'g--', 'm--', 'c--']
    colorEv = ['r+', 'k+', 'g+', 'm+', 'c+']

    for i, tev_i in enumerate(tev): # We iterate over the realizations of the Hawkes Process
        n = -1
        l_t = np.zeros(len(tvec))

        for t in tvec:
            n += 1
            l_t[n] = intensity_hawkes_exp_eval(l_0, alpha_0, w, history=tev_i[tev_i < t], t=t)
            # l_t[n] = l_0 + alpha_0 * np.sum(np.exp(-w * (t - tev[i][tev[i] < t])))

        plt.plot(tvec, l_t, colorLambda[i % len(colorLambda)])
        plt.plot(tev[i], np.zeros(len(tev[i])), colorEv[i % len(colorEv)])

In [ ]:
def preprocess_events(tev, T, w):
    cte_i = np.zeros_like(tev, dtype=float)
    K = 0 # This represents the survival part of the log-likelihood

    for i in range(len(tev)):
        cte_i[i] = np.sum(np.exp(-w * (tev[i] - tev[0:i])))
        K += (1.0 / w) * (1.0 - np.exp(-w * (T - tev[i])))
 
    return cte_i, K

### Play with the Hawkes Process


In [ ]:
# Simulation time
T = 30

# Maximum number of events per realization
maxNev = 1000
# Base intensity
lambda_0 = 1

# Self excitation parameter
alpha_0 = 0.9

# Rate of decay
w = 1

# Number of realizations
Nsamples = 50

In [ ]:
tev = [None] * Nsamples
Tend = [None] * Nsamples
cte_i = [None] * Nsamples
K = np.zeros(Nsamples)
for i in range(Nsamples):
    tev[i], Tend[i] = sample_hawkes(lambda_0, alpha_0, w, T, maxNev)
    cte_i[i], K[i] = preprocess_events(tev[i], Tend[i], w)


In [ ]:
for n, tev_n in enumerate(tev):
    print('Sequence ({}) | len: {}'.format(n, len(tev_n)))

In [ ]:
plot_hawkes(tev, lambda_0, alpha_0, w, T, 10000.0)
plt.ion()  # Make the plot interactive
plt.show()  # Show the plot. May not be needed in IPython

Find the ML estimate of the parameters

In [ ]:
def hawkes_log_lik(T, alpha_opt, lambda_opt, cte_i, K, for_cvx=False):
    # The implementation has to be different for CVX and numpy versions because
    # CVX variables cannot handle the vectorized operations of Numpy  like
    # np.sum and np.log.

    L = 0
    for i in range(len(cte_i)):
        if for_cvx and len(cte_i) > 0:
            L += CVX.sum(CVX.log(lambda_opt + alpha_opt * cte_i[i]))
        else:
            L += np.sum(np.log(lambda_opt + alpha_opt * cte_i[i]))

        L -= lambda_opt * T[i] + alpha_opt * K[i]

    return L

In [ ]:
alpha_opt = CVX.Variable() if alpha_0 > 0 else 0
constraints = [alpha_opt >= 0] if alpha_0 > 0 else []
lambda_opt = CVX.Variable()
constraints.append(lambda_opt >= 0)

complete_log_lik = hawkes_log_lik(Tend,
                                alpha_opt,
                                lambda_opt,
                                cte_i,
                                K,
                                for_cvx=True)
prob = CVX.Problem(
    CVX.Maximize(complete_log_lik),
    constraints=constraints)

result = prob.solve(verbose=True)

In [ ]:
error_alpha = (alpha_opt.value - alpha_0) if alpha_0 > 0 else 0
error_lambda_0 = (lambda_opt.value - lambda_0)

print('error_alpha = {}, error_lambda_0 = {}'
      .format(error_alpha, error_lambda_0))

In [ ]:
print('lambda_0 real: {} | alpha real: {}'.format(lambda_0, alpha_0))
print('lambda_0 est: {} | alpha est: {}'.format(float(lambda_opt.value), float(alpha_opt.value)))
